In [11]:
!pip install pandas sqlalchemy psycopg2-binary

In [34]:
import os
import pandas as pd
import chardet

# Путь к папке с исходными файлами
source_folder = r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_2"
# Путь к папке для чистых файлов
target_folder = r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_clean"

# Создаем папку для чистых файлов, если её нет
os.makedirs(target_folder, exist_ok=True)

# Функция для определения кодировки файла
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read(100000))  # Читаем первые 100кб
    return result['encoding']

# Проходим по всем CSV-файлам в папке
for root, _, files in os.walk(source_folder):
    for file in files:
        if file.endswith(".csv"):
            source_path = os.path.join(root, file)
            relative_path = os.path.relpath(source_path, source_folder)
            target_path = os.path.join(target_folder, relative_path)

            os.makedirs(os.path.dirname(target_path), exist_ok=True)


In [32]:
import os
import pandas as pd
from sqlalchemy import create_engine

# --- Настройки подключения ---
USER = "postgres"
PASSWORD = "123"
HOST = "localhost"
PORT = 5432

# Функция для загрузки файлов из папки в соответствующую БД
def load_csv_to_postgres(folder_path, db_name):
    engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{db_name}")

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            table_name = file.replace("_bom.csv", "")
            csv_path = os.path.join(folder_path, file)
            
            print(f"📥 Загружаем {table_name} → {db_name}...")

            try:
                df = pd.read_csv(csv_path, encoding="utf-8")
            except UnicodeDecodeError:
                print(f"⚠️ Ошибка чтения {file} с utf-8. Пробуем cp1251...")
                df = pd.read_csv(csv_path, encoding="cp1251")

            df.to_sql(table_name, engine, if_exists="replace", index=False)
            print(f"✅ Готово: {table_name}")

# --- Папки с файлами ---
var_dir = r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_2\var"
med_dir = r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_2\med"

# --- Загружаем VAR-файлы ---
load_csv_to_postgres(var_dir, "project_var_2")

# --- Загружаем MED-файлы ---
load_csv_to_postgres(med_dir, "project_mediana_2")


📥 Загружаем context_var → project_var_2...


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 61: invalid continuation byte

In [24]:
import os
import pandas as pd
from sqlalchemy import create_engine

# --- Настройки подключения ---
USER = "postgres"
PASSWORD = "123"
HOST = "localhost"
PORT = 5432

# Папки с файлами
var_dir = r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_2\var"
med_dir = r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_2\med"

# Функция для загрузки файлов из папки в соответствующую БД
def load_csv_to_postgres(folder_path, db_name, suffix):
    engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{db_name}")

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            table_name = file.replace("_bom.csv", "")
            csv_path = os.path.join(folder_path, file)
            
            print(f"📥 Загружаем {table_name} → {db_name}...")
            df = pd.read_csv(csv_path)
            df.to_sql(table_name, engine, if_exists="replace", index=False)
            print(f"✅ Готово: {table_name}")

# --- Загружаем VAR-файлы ---
load_csv_to_postgres(var_dir, "project_var_2", "_var")

# --- Загружаем MED-файлы ---
load_csv_to_postgres(med_dir, "project_mediana_2", "_med")


📥 Загружаем context_var → project_var_2...


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 61: invalid continuation byte

In [20]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

# Настройки подключения
USER = "postgres"
PASSWORD = "123"
HOST = "localhost"
PORT = 5432

# Папки и базы
checklist = [
    (r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_2\var", "project_var_2"),
    (r"D:\data analytics\аналитика 6\powerBI\проект\csv_export_2\med", "project_mediana_2"),
]

# Список с результатами
results = []

for folder, db in checklist:
    engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{db}")
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            table_name = file.replace("_bom.csv", "")
            csv_path = os.path.join(folder, file)

            df = pd.read_csv(csv_path)
            csv_count = len(df)

            with engine.connect() as conn:
                db_count = conn.execute(text(f"SELECT COUNT(*) FROM {table_name}")).scalar()
            
            status = "✅ Совпадает" if csv_count == db_count else "⚠️ Несовпадение"
            results.append({
                "БД": db,
                "Таблица": table_name,
                "CSV-строк": csv_count,
                "DB-строк": db_count,
                "Статус": status
            })

# Выводим как DataFrame
results_df = pd.DataFrame(results)
results_df


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 61: invalid continuation byte